In [ ]:
import numpy as np
import socket
import time
import pickle
import threading
from functools import reduce
from Grouping import *
import warnings
warnings.filterwarnings(action='ignore')

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
port = 4007
sock.bind(('203.255.56.63',port))
sock.listen(1)

def frag(l):
    n = 1
    while 1:
        if l == 0:
            return 0
        if l/n >= 60000:
            n += 1
        else:
            return n

def indexing_mp4(v_path):
    n = len(v_path)
    if n <= 1:
        with open(v_path[0], 'rb') as v:
            v_data = v.read()
            return v_data, v_data.find(b'mdat')

    # 영상들 읽어오기
    v_data = []
    for i in range(n):
        with open(v_path[i], 'rb') as v:
            v_data.append(v.read())

    # 영상들 헤더 위치 저장
    header_idx = []
    for i in range(n):
        header_idx.append(v_data[i].find(b'mdat'))
    
    # 영상들 바이너리 형식으로 읽어오기
    for i in range(n):
        v_data[i] = np.frombuffer(v_data[i], dtype='uint8')
        
    def XOR(data1, data2):
        l1 = len(data1)
        l2 = len(data2)
        
        if l1 > l2:
            xor_len = l2
            add_data = data1[l2:l1]
        else:
            xor_len = l1
            add_data = data2[l1:l2]

        xor_v = np.bitwise_xor(data1[:xor_len], data2[:xor_len])
        xor_data = np.append(xor_v, add_data)

        return xor_data
    
    xor_data = reduce(XOR, v_data)
    xor_data = xor_data.tostring()
    return xor_data, max(header_idx) + 5

def Request(num):
    global c_n, clients, addrs, r_v, c_list, st
    client, addr = sock.accept()
    
    if len(c_list) == 0:    # 처음으로 인식
        num = 1
        
    clients.append([client])
    addrs.append(addr)
    c_n[client] = num
    
    t = threading.Thread(target=Request, args=(num+1,))
    t.start()
    
    print('Connected to {}'.format(addr))
    client.send(str(num).encode())  # 클라이언트 번호 보내기
    
    r_v.append(client.recv(32).decode())
    c_list.append(pickle.loads(client.recv(1024)))
    
def multicast(clients, v_list, ip, port):
    MCAST_GRP = ip
    MCAST_PORT = port
    MULTICAST_TTL = 32
    
    for C in clients:
        C.send('MC'.encode())
        time.sleep(0.3)
        C.send(ip.encode())
        time.sleep(0.3)
        C.send(str(port).encode())
        time.sleep(0.3)
    
    time.sleep(0.3)
    server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
    server.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, MULTICAST_TTL)
    
    server.sendto(pickle.dumps(v_list), (MCAST_GRP, MCAST_PORT))
    time.sleep(0.3)
    n = len(v_list)

    v_path = []
    for i in v_list:
        v_path.append('Videos/{}'.format(i))

    data, header = indexing_mp4(v_path)
    l = len(data[header:])

    n_h = frag(header)
    n_d = frag(l)

    server.sendto(str(n_h).encode(), (MCAST_GRP, MCAST_PORT))
    time.sleep(0.3)
    server.sendto(str(n_d).encode(), (MCAST_GRP, MCAST_PORT))
    time.sleep(0.3)

    for i in range(n_h):
        server.sendto(data[i*header//n_h : (i+1)*header//n_h], (MCAST_GRP, MCAST_PORT))
        time.sleep(0.7)

    for i in range(n_d):
        server.sendto(data[header + i*l//n_d : header + (i+1)*l//n_d], (MCAST_GRP, MCAST_PORT))
        time.sleep(0.7)
    server.close()

def unicast(Clients, client, addr, r_v):
    client.send('UC'.encode())
    
    idx = Clients.index([client])
    client.send(str(addr[1]).encode())
    time.sleep(0.3)
    
    server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    v_path = 'Videos/{}'.format(r_v[0])
    with open(v_path, 'rb') as v:
        data = v.read()
    
    header = data.find(b'mdat') + 5
    l = len(data[header:])
    
    n_h = frag(header)
    n_d = frag(l)
    
    server.sendto(str(n_h).encode(), addr)
    time.sleep(0.3)
    server.sendto(str(n_d).encode(), addr)
    time.sleep(0.3)
    
    for i in range(n_h):
        server.sendto(data[i*header//n_h : (i+1)*header//n_h], addr)
        time.sleep(0.7)
    
    for i in range(n_d):
        server.sendto(data[header + i*l//n_d : header + (i+1)*l//n_d], addr)
        time.sleep(0.7)
    
def Transmission(C_N, Clients, Addrs, R_V):
    global MCAST_IP, MCAST_PORT
    for i in range(len(Clients)):
        if len(Clients[i]) == 1:
            t = threading.Thread(target=unicast, args=(Clients, Clients[i][0], Addrs[i], R_V[i]))
            t.start()
            uc_client = ['Clients[{}]'.format(C_N[Clients[i][0]])]
            print('UC {} to {}'.format(R_V[i], uc_client))
        else:
            t = threading.Thread(target=multicast, args=(Clients[i], R_V[i], MCAST_IP, MCAST_PORT))
            t.start()
            mc_clients = ['Client[{}]'.format(C_N[k]) for k in Clients[i]]
            if len(R_V[i]) == 1:
                print('MC {} to {}'.format(R_V[i], mc_clients))
            else:
                print('XMC {} to {}'.format(R_V[i], mc_clients))
            MCAST_PORT += 1
    if len(clients):
        print('--------------------------------------------------------------------------------------')
    
MCAST_IP = '224.1.1.1'
MCAST_PORT = 5007

waiting = 3
c_n = {}
clients, addrs = [], []
r_v, c_list = [], []

t = threading.Thread(target=Request, args=(1,))
t.start()
while 1:
    if len(clients) == 0:
        time.sleep(1)
        continue
    
    time.sleep(waiting)
    
    num_clients = len(clients)
    V = list(set(r_v))
    
    clients, r_v, c_list = BCG(V, clients, r_v, c_list)
    clients, r_v, c_list = XBCG(V, clients, r_v, c_list)
    
    t = threading.Thread(target=Transmission, args=(c_n, clients, addrs, r_v))
    t.start()
    
    c_n = {}
    clients, addrs = [], []
    r_v, c_list = [], []

Connected to ('203.255.56.63', 56420)
Connected to ('203.255.56.63', 56427)
XMC ['Beach7.mp4', 'Light6.mp4'] to ['Client[1]', 'Client[2]']
Connected to ('203.255.56.63', 56434)
UC ['Beach5.mp4'] to ['Clients[1]']
Connected to ('203.255.56.63', 56443)
UC ['Beach8.mp4'] to ['Clients[1]']
Connected to ('203.255.56.63', 56451)
UC ['Stars6.mp4'] to ['Clients[1]']
Connected to ('203.255.56.63', 56456)
Connected to ('203.255.56.63', 56461)
